In [1]:
import pandas as pd
import numpy as np

In [2]:
#read the following csv file and only that specific cloumn
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
rating_df=pd.read_csv('ratings.csv',usecols=['userId', 'movieId', 'rating'],dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})



In [3]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [3]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [5]:
#so some movies has higher no. of rating
df.groupby(by ="title").count().loc["Toy Story (1995)"]

userId     215
movieId    215
rating     215
Name: Toy Story (1995), dtype: int64

In [6]:
#drop rows which dont have title
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

In [7]:
#groupby title and count the total rating given...then reset inddx and rename rating as total rating
movie_ratingCount = (combine_movie_rating.groupby(by = ['title'])['rating'].count().reset_index().rename(columns = {'rating': 'totalRatingCount'}))
movie_ratingCount.head()


,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [8]:
#merge main data frame with rating count
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [10]:
#on an average each movie has 10 ratings with 22 S.D
print(movie_ratingCount['totalRatingCount'].describe())

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: totalRatingCount, dtype: float64


In [13]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount[rating_with_totalRatingCount["totalRatingCount"]>=popularity_threshold]
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [14]:
#final data frame with movies haviing more than 50 rating
rating_popular_movie.shape

(41362, 5)

In [15]:
## First lets create a Pivot matrix

movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()
#total 450 movies

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [16]:
#this is the way we will fetch index of any movie we want to get recommendation for
#below given query index is random
test = movie_features_df.reset_index()
test[test["title"] == "300 (2007)"].index[0]

4

In [17]:
movie_features_df.values

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 3. , 0. , 4.5],
       ...,
       [5. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 3. , 0. , ..., 0. , 0. , 3.5],
       [0. , 0. , 0. , ..., 3. , 0. , 4. ]], dtype=float32)

In [20]:
#there are 450 movies 
movie_features_df.shape

(450, 606)

In [18]:
from scipy.sparse import csr_matrix
#converting pivot table into matrix
#only ussed to fit in KNN model
movie_features_df_matrix = csr_matrix(movie_features_df.values)
movie_features_df_matrix

<450x606 sparse matrix of type '<class 'numpy.float32'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [19]:

from sklearn.neighbors import NearestNeighbors

#this will fit item to item cosine distance
#this is neibhours not k neibhours.....so we will define neibhours after that
#cosine is cos theta of angle between two vectors
#brute will compare each vector to every other vector
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [26]:
#fetch specific index data ...which is row with index as movie id...take values only....then reshape it to (1,-1)
#original values are just list of 606 values, reshape(1,-1) makes it a vector 0f 1X606
movie_features_df.iloc[346].values.reshape(1,-1).shape

(1, 606)

## ITEM TO ITEM TYPE COLLABORATIVE FILTERING

In [28]:
def recommend(movie,n):
    #fetching query index
    test = movie_features_df.reset_index()
    #return index for movie query
    query_index = test[test["title"] == movie].index[0]
    #fetching nearest distance and indices of closest n+1 movies because one movie will be repeated
    #it return two array...one is distacne and other is indices
    distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index].values.reshape(1, -1), n_neighbors = n+1)
    #distances.flatten since only disctances is a list inside list
    for i in range(0, len(distances.flatten())):
        #skip first movie as it is same as given movie
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
    

In [29]:
recommend('10 Things I Hate About You (1999)',10)

Recommendations for 10 Things I Hate About You (1999):

1: Wedding Singer, The (1998), with distance of 0.5506574511528015:
2: Grease (1978), with distance of 0.571800708770752:
3: Bridget Jones's Diary (2001), with distance of 0.5761856436729431:
4: Legally Blonde (2001), with distance of 0.5950943827629089:
5: Clueless (1995), with distance of 0.6036181449890137:
6: What Women Want (2000), with distance of 0.605096161365509:
7: Hook (1991), with distance of 0.6091365814208984:
8: Charlie's Angels (2000), with distance of 0.6249631643295288:
9: Moulin Rouge (2001), with distance of 0.625316858291626:
10: Almost Famous (2000), with distance of 0.6315915584564209:


## AUTOSUGGESTION FOR SPELLING 

In [31]:
import difflib

case_insensitive_movies_list = [i.lower() for i in movie_features_df.index]


In [34]:
def find_top_10_matches(incomplete_spelling,case_insensitive_movies_list):
    # Create a SequenceMatcher object to find the closest matches
    #The basic idea is to find the longest contiguous matching subsequence. a ratio above 0.6 is a higly likelly match
    matcher = difflib.SequenceMatcher()

    # Initialize a list to store the matches and their ratios
    matches = []

    # Iterate through the word list
    for word in case_insensitive_movies_list:
        # Set the sequences for the matcher (incomplete spelling and the word)
        matcher.set_seqs(incomplete_spelling, word)

        # Get the matching ratio
        ratio = matcher.ratio()

        # Append the word and ratio to the list of matches
        matches.append((word, ratio))

    # Sort the matches by ratio in descending order
    matches.sort(key=lambda x: x[1], reverse=True)

    # Get the top 10 matches
    top_10_matches = matches[:10]
    #print("your spelling donot matches with the record...here are some suggestion...please copy from them")
    return top_10_matches


## MERGING WITH MAIN CODE

In [35]:
def bestrecommend(movie,n):
    
    if movie in movie_features_df.index:
        #fetching query index
        test = movie_features_df.reset_index()
        query_index = test[test["title"] == movie].index[0]
         #fetching nearest distance and indices of closest n+1 movies because one movie will be repeated
         #it return two array...one is distacne and other is indices
        distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index].values.reshape(1, -1), n_neighbors = n+1)
        for i in range(0, len(distances.flatten())):
            if i == 0:
                print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
            else:
                print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
    else:
        test=find_top_10_matches(movie,case_insensitive_movies_list)
        print("your given name is not in the list...here are some best matches..please copy from that")
        test2 = [i[0] for i in test]
        print(test2)
    

In [36]:
bestrecommend("batman",4)

your given name is not in the list...here are some best matches..please copy from that
['batman (1989)', 'batman begins (2005)', 'batman forever (1995)', 'batman returns (1992)', 'rain man (1988)', 'what women want (2000)', 'armageddon (1998)', 'beautiful mind, a (2001)', 'american pie (1999)', 'pretty woman (1990)']


# USER TO USER


In [37]:
user_to_movie_df=rating_popular_movie.pivot_table(index='userId',columns='title',values='rating').fillna(0)
user_to_movie_df.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),...,Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,5.0,5.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
4,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
user_to_movie_df.shape

(606, 450)

In [39]:
# transform matrix to scipy sparse matrix
user_to_movie_sparse_df = csr_matrix(user_to_movie_df.values)
user_to_movie_sparse_df

<606x450 sparse matrix of type '<class 'numpy.float32'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [40]:
knn_model2 = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model2.fit(user_to_movie_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [43]:
def getsimilaruser(user,n):
    test = user_to_movie_df.reset_index()
    query_index = test[test["userId"] == user].index[0]
    #fetching nearest distance and indices of closest n+1 users because one movie will be repeated
    distances, indices = knn_model2.kneighbors(user_to_movie_df.iloc[query_index].values.reshape(1, -1), n_neighbors = n+1)
    test2 = zip(distances, indices)     
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for user {0}:\n'.format(user_to_movie_df.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, user_to_movie_df.index[indices.flatten()[i]], distances.flatten()[i]))
    return test2     
   


In [44]:
getsimilaruser(303,7)

Recommendations for user 303:

1: 149, with distance of 0.5113699436187744:
2: 265, with distance of 0.5479335784912109:
3: 95, with distance of 0.5517808198928833:
4: 164, with distance of 0.5555366277694702:
5: 312, with distance of 0.5618977546691895:
6: 267, with distance of 0.5718188285827637:
7: 186, with distance of 0.5899441242218018:


In [95]:
def weightedmovie_recom(user,n):
    test = user_to_movie_df.reset_index()
    query_index = test[test["userId"] == user].index[0]
    
    #fetching nearest distance and indices of closest n+1 users because one movie will be repeated
    distances, indices = knn_model2.kneighbors(user_to_movie_df.iloc[query_index].values.reshape(1, -1), n_neighbors = n+1)
    
    #we addes zero in both indices and distances as they are list inisde list
    #ignore first suggestion as it is same as input
    distances = distances[0][1:]
    indices = indices[0][1:]
    
    #below weight list gives large weight to close disctance vectors
    weightage_listx = 1 - distances/np.sum(distances)
    
    
    #fetch vector corresponding to user identified as similar user
    mov_rtngs_sim_users = user_to_movie_df.values[indices]
    movies_list = user_to_movie_df.columns
    
    
    #produce a weight list of len(weightage_listx)x450 matrix
    weightage_list1 = weightage_listx[:,np.newaxis] + np.zeros(len(movies_list))
  
    #element by element multiplication of two matrices both shape len(weightage_listx)x450
    #it produces rating given by similar users but weighted 
    new_rating_matrix = weightage_list1*mov_rtngs_sim_users
    
    #and finally a single array for rating of all movies
    mean_rating_list = new_rating_matrix.sum(axis =0)
    
    #argsort gives an array which sort the provided list
    sorted_indices = np.argsort(mean_rating_list)[::-1]
    
    #we use sorted indices to sort the original movie list and take first n
    final_list =list(movies_list[sorted_indices[:n]])
    
    return final_list
    
    
    

In [99]:
weightedmovie_recom(203,6)

['Shawshank Redemption, The (1994)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Matrix, The (1999)',
 'Forrest Gump (1994)']

## Remove Movie Already scene

In [100]:
def weightedmovie_recom_new(user,n):
    test = user_to_movie_df.reset_index()
    query_index = test[test["userId"] == user].index[0]
    
    #fetching nearest distance and indices of closest n+1 users because one movie will be repeated
    distances, indices = knn_model2.kneighbors(user_to_movie_df.iloc[query_index].values.reshape(1, -1), n_neighbors = n+1)
    
    #we addes zero in both indices and distances as they are list inisde list
    #ignore first suggestion as it is same as input
    distances = distances[0][1:]
    indices = indices[0][1:]
    
    #below weight list gives large weight to close disctance vectors
    weightage_listx = 1 - distances/np.sum(distances)
    
    
    #fetch vector corresponding to user identified as similar user
    mov_rtngs_sim_users = user_to_movie_df.values[indices]
    movies_list = user_to_movie_df.columns
    
    
    #produce a weight list of len(weightage_listx)x450 matrix
    weightage_list1 = weightage_listx[:,np.newaxis] + np.zeros(len(movies_list))
  
    #element by element multiplication of two matrices both shape len(weightage_listx)x450
    #it produces rating given by similar users but weighted 
    new_rating_matrix = weightage_list1*mov_rtngs_sim_users
    
    #and finally a single array for rating of all movies
    mean_rating_list = new_rating_matrix.sum(axis =0)
    
    #argsort gives an array which sort the provided list
    sorted_indices = np.argsort(mean_rating_list)[::-1]
    
    #array of user given rating to be excluded
    userrating = np.array(user_to_movie_df.loc[user])
    
    #get indices for user rating in decreasing order , then count only rating given and finally lenth of it
    #it gives total movie scence by user
    total_movie_scene = (userrating[np.argsort(userrating)[::-1]]>0).sum()
    
    #use same idences to sort movies and take first test2 movies
    moviealreadyseen = movies_list[np.argsort(userrating)[::-1]][:total_movie_scene]
    
    count = 0
    finallist=[]
    for i in movies_list[sorted_indices]:
            if len(finallist) == n:
                break
            elif i in moviealreadyseen:
                continue   
            else:
                finallist.append(i)
    
    
    return finallist
    
    

In [101]:
weightedmovie_recom_new(203,6)

['Forrest Gump (1994)',
 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
 'Fight Club (1999)',
 'Sherlock Holmes (2009)',
 "Pirates of the Caribbean: Dead Man's Chest (2006)",
 'Finding Nemo (2003)']